In [4]:
import pandas as pd
import requests

import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import glob
import os

In [3]:
import re

def get_key_from_environment(file_path: str, key: str) -> str | None:
    """
    Extracts a key from Angular's environment.ts file.

    Args:
        file_path: Path to environment.ts
        key: The key name (e.g., "apiKey")

    Returns:
        The value as a string, or None if not found.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Regex to match key: 'value' or key: "value"
    pattern = rf'{key}\s*:\s*[\'"]([^\'"]+)[\'"]'
    match = re.search(pattern, content)

    return match.group(1) if match else None


# Example usage
file_path = "../src/environments/environment.ts"
api_key = get_key_from_environment(file_path, "apiToken")

print("API Key:", api_key)


API Key: 55a064fc7f01134ea1d951c30a610354


In [7]:
header = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

In [10]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

scale = 1
start_date = datetime(2024, 9, 1)   # Sept 2024
end_date = datetime(2025, 8, 1)     # Aug 2025

# Loop over months
date = start_date
while date <= end_date:
    date_str = date.strftime("%Y-%m")
    url = f"https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale{scale:03d}&date={date_str}"
    
    print(f"Fetching {url} ...")
    res = requests.get(url, headers=header)
    
    if res.status_code == 200:
        file = f"./data/spi{scale:03d}_{date_str}.tif"
        with open(file, "wb") as f:
            f.write(res.content)
    
    # Move to next month
    date += relativedelta(months=1)

Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2024-09 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2024-10 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2024-11 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2024-12 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2025-01 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2025-02 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2025-03 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2025-04 ...
Fetching https://api.hcdp.ikewai.org/raster?datatype=spi&period=month&timescale=timescale001&date=2025-05 ...
Fetching h

In [ ]:


shapefile = "../public/hawaii_climate_divisions.shp"

raster_folder = "./data"

gdf = gpd.read_file(shapefile)

id_col = "name"  

records = []

for tif in sorted(glob.glob(os.path.join(raster_folder, "spi*.tif"))):
    # Extract date from filename (spi001_2024-09.tif → 2024-09)
    date = os.path.basename(tif).split("_")[1].replace(".tif", "")
    
    # Zonal statistics
    stats = zonal_stats(
        vectors=gdf,
        raster=tif,
        stats=["mean"],
        geojson_out=False,
        nodata=-9999
    )
    
    # Store results
    for div, s in zip(gdf[id_col], stats):
        records.append({
            "division": div,
            "date": date,
            "mean_spi": s["mean"]
        })

df = pd.DataFrame(records)
df = df.pivot(index="division", columns="date", values="mean_spi")
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv("division_spi_timeseries.csv")



In [27]:
df

date,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08
division,,,,,,,,,,,,
Hawaiʻi Mauka,0.038066,0.065666,1.049135,-1.860736,0.920774,-1.160019,0.075928,1.234646,0.205015,0.197425,-0.308166,-0.507025
Hilo,0.168698,-0.036994,0.462315,-2.008300,-0.166594,-1.327917,-0.284606,-2.156249,-0.406141,-0.043608,-0.844669,-1.233752
Kaʻu,0.549397,0.128859,0.878572,-1.457877,0.471739,-0.980069,0.016767,0.146130,0.118653,0.250923,-0.198298,-0.368525
Kona,0.301276,-0.038568,0.953837,-1.515916,1.546793,-0.967202,-0.057603,1.056530,-0.036059,1.278930,-0.396075,-0.510286
Koʻolau,-0.826163,0.049489,-0.645679,-1.493716,1.147098,-1.173091,-0.401305,1.286310,0.401921,-0.886988,-0.346858,-2.181717
Leeward Kauaʻi,-0.684435,-0.484060,-1.303766,-1.091496,0.245709,-0.072747,-0.880396,0.985224,0.511431,-0.064478,-0.489653,-1.416167
Leeward Kohala,-0.387270,0.183305,0.219875,-1.166898,1.077449,-1.027650,-0.063557,0.683993,-0.079663,0.471994,-0.217988,-0.369120
Leeward Maui Nui,-0.659705,-0.305836,-0.774408,-1.477284,1.092026,-1.045730,-0.952339,0.653340,-0.658746,-0.645195,-0.712859,-0.596300
Waianae,-1.346182,-0.602371,-0.512149,-1.456255,1.189836,-0.945446,-0.649065,0.844288,0.359477,-0.238175,-0.379399,-1.498299


In [36]:
shapefile = "../public/Coastline.shp"

raster_folder = "./data"

gdf = gpd.read_file(shapefile)

id_col = "isle"  

records = []

for tif in sorted(glob.glob(os.path.join(raster_folder, "spi*.tif"))):
    # Extract date from filename (spi001_2024-09.tif → 2024-09)
    date = os.path.basename(tif).split("_")[1].replace(".tif", "")
    
    # Zonal statistics
    stats = zonal_stats(
        vectors=gdf,
        raster=tif,
        stats=["mean"],
        geojson_out=False,
        nodata=-9999
    )
    
    # Store results
    for div, s in zip(gdf[id_col], stats):
        records.append({
            "division": div,
            "date": date,
            "mean_spi": s["mean"]
        })

df = pd.DataFrame(records)
df = df.pivot(index="division", columns="date", values="mean_spi")
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv("../public/island_spi_timeseries.csv")
print(df)


date        2024-09   2024-10   2024-11   2024-12   2025-01   2025-02  \
division                                                                
Hawaiʻi    0.164475  0.039084  0.615427 -1.762296  0.516474 -1.215282   
Kahoolawe -0.560859 -0.067711 -0.315173 -0.980359  1.056914 -0.649988   
Kauaʻi    -0.738852 -0.171122 -0.914319 -1.164958  0.180476 -0.874774   
Lānaʻi    -0.370052 -0.245773 -0.757944 -1.625310  0.879548 -1.001471   
Maui      -0.602870 -0.045850 -0.911720 -1.758803  1.194903 -1.549807   
Molokaʻi  -0.960090 -0.435848 -0.405934 -1.657838  0.940091 -1.092378   
Oʻahu     -1.126451 -0.328286 -0.569652 -1.470650  1.172144 -1.042343   

date        2025-03   2025-04   2025-05   2025-06   2025-07   2025-08  
division                                                               
Hawaiʻi   -0.105839 -0.209279 -0.126939  0.233570 -0.522495 -0.758712  
Kahoolawe -0.726587  2.042276 -0.700550  0.171366 -0.556886 -0.535833  
Kauaʻi    -0.754501  1.323397  0.633320  0.085576 -0.3

In [5]:
shapefile = "../public/Coastline.shp"
raster_folder = "./data"

# Load shapefile and dissolve to one feature (statewide)
gdf = gpd.read_file(shapefile)
gdf_statewide = gdf.dissolve()  # merges all polygons into one

records = []

for tif in sorted(glob.glob(os.path.join(raster_folder, "spi*.tif"))):
    # Extract date from filename (spi001_2024-09.tif → 2024-09)
    date = os.path.basename(tif).split("_")[1].replace(".tif", "")
    
    # Compute zonal stats for the dissolved geometry
    stats = zonal_stats(
        vectors=gdf_statewide,
        raster=tif,
        stats=["mean"],
        geojson_out=False,
        nodata=-9999
    )
    
    records.append({
        "date": date,
        "value": stats[0]["mean"]
    })


# Pivot to wide format: one row (statewide), months as columns
df = pd.DataFrame(records).set_index("date").T
df.insert(0, "state", "statewide")  # add proper state column
df.to_csv("../public/statewide_spi_timeseries.csv", index=False)

print(df)

date       state   2024-09   2024-10   2024-11   2024-12   2025-01   2025-02  \
value  statewide -0.185488 -0.050791  0.115566 -1.668572  0.656251 -1.193424   

date    2025-03   2025-04   2025-05   2025-06   2025-07   2025-08  
value -0.371624  0.136254 -0.114843 -0.023033 -0.532988 -0.950657  
